In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import tensorlayer 
import shutil
import pathlib
import skimage
from skimage import transform 
from skimage.io import imread_collection
from skimage.color import rgb2gray
from skimage.io import imread_collection
AUTOTUNE = tf.data.experimental.AUTOTUNE


In [2]:
#Insert path to folder to store best model
path_models = "" 

#Insert path to folder with test data
path_images_train = ""

#Insert path to folder with train data
path_images_test = ""

#Insert path to folder with new data
path_images_new=""

In [3]:
#Preprocessing

def get_data(image_directory):
    dirs = [d for d in os.listdir(image_directory) 
                   if os.path.isdir(os.path.join(image_directory, d))]
    labels = []
    images = []
    for d in dirs:
        label_directory = os.path.join(image_directory, d)
        file_names = [os.path.join(label_directory, f)
                      for f in os.listdir(label_directory)]
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    return images, labels


train_image_directory = path_images_train
test_image_directory = path_images_test

train_images, train_labels = get_data(train_image_directory)
test_images, test_labels = get_data(test_image_directory)

In [10]:
def transform_existing_data(data):
    if data[0][0].shape[-1]==3:
        print('Converting image to black and white')
        data[0] = rgb2gray(data)
    images28 = [transform.resize(image, (28, 28)) for image in data[0]]
    images28 = np.array(images28)
    images28 = np.stack([i.flatten() for i in images28], axis=0)
    labels_one_hot=np.eye(len(set(data[1])))[data[1]]
    order = np.arange(images28.shape[0])
    np.random.shuffle(order)
    return images28[order], labels_one_hot[order]

In [11]:
def transform_new_data(image_array):
    new_images = []
    for image in image_array:
        if image.shape[-1]==3:
            print('Converting image to black and white')
            new_images.append(rgb2gray(image))
    images28 = [transform.resize(image, (28, 28)) for image in new_images]
    images28 = np.array(images28)
    images28 = np.stack([i.flatten() for i in images28], axis=0)
    return images28

In [12]:
images_train, labels_train = transform_existing_data([train_images, train_labels])
images_test, labels_test = transform_existing_data([test_images, test_labels])

In [20]:
#Creating and training Neural Network

def set_weights(var, shape):
    initializer = tf.initializers.variance_scaling(scale=2.0, mode='fan_in', distribution='truncated_normal',
                                             seed=None, dtype=tf.float32) 
    return tf.get_variable(var, shape=shape, initializer=initializer)

def bias_element(shape):
    init = tf.constant(0.1, shape=shape)
    return tf.Variable(init)

def convolutional_layer2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def pooling_layer_2_on_2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')



In [ ]:
x = tf.placeholder(tf.float32, [None, 28*28])
y_= tf.placeholder(tf.float32, [None, 10])
x_img = tf.reshape(x, [-1,28,28,1])

#Layer №1
W_convolutional_L1 = set_weights("W1", [5,5,1,32])
b_convolutional_L1 = bias_element([32])
h_convolutional_L1 = tf.nn.relu(convolutional_layer2d(x_img, W_convolutional_L1) + b_convolutional_L1)
h_pool1 = pooling_layer_2_on_2(h_convolutional_L1)

#Layer №2
W_convolutional_L2 = set_weights("W2", [5,5,32,64])
b_convolutional_L2 = bias_element([64])
h_convolutional_L2 = tf.nn.relu(convolutional_layer2d(h_pool1, W_convolutional_L2) + b_convolutional_L2)
h_pool2 = pooling_layer_2_on_2(h_convolutional_L2)

#Layer №3
W_full_connection1 = set_weights("W3", [7*7*64, 1024])
b_full_connection1 = bias_element([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1,7*7*64])
h_full_connection1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_full_connection1) + b_full_connection1)


keep_prob = tf.placeholder(tf.float32)
dropout = tf.nn.dropout(h_full_connection1, keep_prob)

#Layer №4
W_full_connection2 = set_weights("W5", [1024, 10])
b_full_connection2 = bias_element([10])
y_conv = tf.matmul(dropout, W_full_connection2) + b_full_connection2

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

In [23]:
#Optimizers and parameters choice
global All_optimizers 
All_optimizers= [(tf.train.GradientDescentOptimizer(0.001).minimize(cross_entropy),'SGD'),
(tf.train.MomentumOptimizer(0.001, momentum=0.9, use_nesterov=False).minimize(cross_entropy),'Momentum'),
(tf.train.MomentumOptimizer(0.001, momentum=0.9, use_nesterov=True).minimize(cross_entropy),'Nesterov_Momentum'),
(tf.train.RMSPropOptimizer(0.001).minimize(cross_entropy),'RMSprop'),
(tf.train.AdamOptimizer(0.001).minimize(cross_entropy),'Adam'),
(tf.train.AdagradOptimizer(0.001).minimize(cross_entropy),'Adagrad'),
(tf.train.AdadeltaOptimizer(learning_rate = 1).minimize(cross_entropy),'Adadelta'),
(tensorlayer.optimizers.AMSGrad(0.001).minimize(cross_entropy),'AMSgrad')
]

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
#Epochs and batch size choice
training_epochs = 1
batch_size = 32

In [34]:
#NN Training process
def Neural_network_learning(Name_str, optimizer,train_data, train_labels, test_data, test_labels,initializer=tf.global_variables_initializer(),accur_metric=tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1)), tf.float32)), path=path_models):
    with tf.Session() as Learning_process:
        Learning_process.run(initializer)

        print("\nConducting Neural network learning process using optimizer ",Name_str )
        avg_loss = list()
        for epoch in range(1,training_epochs+1):
            for batch in range(0,int(np.floor(train_data.shape[0]/batch_size))):
                batch_xs, batch_ys = train_data[batch*batch_size:(batch*batch_size+batch_size)], train_labels[batch*batch_size:(batch*batch_size+batch_size)]
                _, loss = Learning_process.run([optimizer, cross_entropy], 
                                    feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})
                avg_loss.append(loss)
    
            train_accuracy = accur_metric.eval(feed_dict={x:batch_xs, y_:batch_ys, keep_prob: 1.0})
            print("Epoch %03d, accuracy %.2f"%(epoch, train_accuracy))

        Accuracy = accur_metric.eval(feed_dict={x: test_data, y_: test_labels, keep_prob: 1.0})
        print("Image classification accuracy on test data - %g" %Accuracy)
        save_path = tf.train.Saver().save(Learning_process,"".join([path,Name_str,"/model.ckpt"]))
        return (Name_str,Accuracy,avg_loss)

In [ ]:
Avg_accuracy_and_loss_algorithms = dict()
best_algorithm = All_optimizers[0][1]
for i in range (0,len(All_optimizers)):
    Avg_accuracy_and_loss_algorithms[All_optimizers[i][1]] = Neural_network_learning( All_optimizers[i][1], All_optimizers[i][0], images_train, labels_train, images_test, labels_test)
    if i!=0:
        if Avg_accuracy_and_loss_algorithms[best_algorithm][1] < Avg_accuracy_and_loss_algorithms[All_optimizers[i][1]][1]:
            try:
                shutil.rmtree("".join([path_models,Avg_accuracy_and_loss_algorithms[best_algorithm][0]]))
            except OSError as e:
                print("Deletion error: %s : %s" % ("".join([path_models,Avg_accuracy_and_loss_algorithms[best_algorithm][0]]), e.strerror))
            best_algorithm = All_optimizers[i][1]
        else:
            try:
                shutil.rmtree("".join([path_models,All_optimizers[i][1]]))
            except OSError as e:
                print(Deletion error: %s : %s" % ("".join([path_models,Avg_accuracy_and_loss_algorithms[best_algorithm][0]]), e.strerror))                   
print("\nBest algorithm - %s with accuracy of %.3f"%(best_algorithm, Avg_accuracy_and_loss_algorithms[best_algorithm][1] ))
shutil.move("".join([path_models,best_algorithm]), "".join([path_models,"Best_model"]))

In [ ]:
#Classifying new images

file_names = [f for f in os.listdir(path_images_new) if '.DS_Store' not in f]
new_data_directory = path_images_new
new_directory = new_data_directory+'/*'
new_images = imread_collection(new_directory)
new_images = transform_new_data(new_images)

In [ ]:
predicted_labels = []
with tf.Session() as Predict_session:
    Predict_session.run(tf.global_variables_initializer())
    tf.train.Saver().restore(Predict_session, "".join([path_models,'Best_model',"/model.ckpt"]))
    predicted_labels=Predict_session.run(tf.argmax(y_conv,1),feed_dict={x: new_images, keep_prob: 1.0})
print('Predicted classes:\n')
for i in range(len(predicted_labels)):
    print("Image ",file_names[i], " – ","class '",predicted_labels[i],"'" )